In [4]:
!pip install --upgrade py2neo
!pip install --upgrade neo4j
!pip install --upgrade ipython-cypher
!pip install networkx
!pip install python-igraph
!pip install plotly
!pip install pandas
!pip install --upgrade matplotlib
!pip install jgraph
!pip install neo4jupyter
from py2neo import Graph,Node,Relationship
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

uri = "neo4j+s://********.databases.neo4j.io:****"
user = "neo4j"
pwd = "******************************************"
graph = Graph(uri, auth=(user, pwd))

def run_query_short(purpose, defineX, defineY):
    if purpose == 'confounder':
        query = "MATCH p= (x:construct)<-[r1:causes]-(confounder:construct) -[r2:causes]-> (y:construct) <-[r3:causes]- (x:construct) \
        WHERE x.name ='"+str(defineX)+"' AND y.name ='"+str(defineY)+"' \
        RETURN x.measure, y.measure, confounder.measure"
        return graph.run(query)
    elif purpose == 'IV':
        query = "MATCH p1= (IV:construct)-[r1:causes]->(x:construct)-[r2: causes]->(y:construct) \
        WHERE x.name='"+str(defineX)+"' AND y.name='"+str(defineY)+"' \
        AND NOT EXISTS {(IV:construct)-[]->(y:construct)} \
        RETURN IV.measure, x.measure, y.measure"
        return graph.run(query)
    else:
        return print("You should correctly specify your query inputs.")

def run_query_full(purpose, defineX, defineY):
    if purpose == 'confounder':
        query = "MATCH p= (x:construct)<-[r1:causes]-(confounder:construct) -[r2:causes]-> (y:construct) <-[r3:causes]- (x:construct) \
        WHERE x.name ='"+str(defineX)+"' AND y.name ='"+str(defineY)+"' \
        RETURN x.measure, y.measure, confounder.measure, r3.Curators, r3.SampleSize, r3.SampleCountries, r3.SampleIndustries, \
        r3.Reference, r3.PublicationTitle, r3.Theory, r3.Direction, r3.Moderator"
        return graph.run(query)
    elif purpose == 'IV':
        query = "MATCH p1= (IV:construct)-[r1:causes]->(x:construct)-[r3:causes]->(y:construct) \
        WHERE x.name='"+str(defineX)+"' AND y.name='"+str(defineY)+"' \
        AND NOT EXISTS {(IV:construct)-[]->(y:construct)} \
        RETURN IV.measure, x.measure, y.measure, r3.Curators, r3.SampleSize, r3.SampleCountries, r3.SampleIndustries, \
        r3.Reference, r3.PublicationTitle, r3.Theory, r3.Direction, r3.Moderator"
        return graph.run(query)
    else:
        return print("You should correctly specify your query inputs.")

def run_query(purpose, defineX, defineY, report):
    if report == "short":
        return run_query_short(purpose, defineX, defineY)
    elif report == "full":
        return run_query_full(purpose, defineX, defineY)
    else:
        return print("You should correctly specify your query reporting format [short/full].")

In [5]:
# Determine if a variable X is endogeneous for variable Y, if so, return confounders

purpose = 'confounder' # write 'confounder' or 'IV' to launch the search
defineX = 'economic reward' # write the name of the construct X
defineY = 'job performance' # write the name of construct Y
report ='full' # write 'short' or 'full' for the report format
run_query(purpose, defineX, defineY, report)

x.measure,y.measure,confounder.measure,r3.Curators,r3.SampleSize,r3.SampleCountries,r3.SampleIndustries,r3.Reference,r3.PublicationTitle,r3.Theory,r3.Direction,r3.Moderator
internal pay ratio,absenteeism,external pay ratio,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",1462,Italy,Manufacturing and Service,TPS15HR,Human Relations,Equity theory,Unclear,performance-related pay policies
internal pay ratio,absenteeism,external pay ratio,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",1462,Italy,Manufacturing and Service,TPS15HR,Human Relations,Equity theory,Unclear,performance-related pay policies
internal pay ratio,absenteeism,external pay ratio,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",1462,Italy,Manufacturing and Service,TPS15HR,Human Relations,Equity theory,Unclear,performance-related pay policies


In [6]:
# Return an instrumental variable (IV) for an independent variable x, given dependent variable y

purpose = 'IV' # write 'confounder' or 'IV' to launch the search
defineX = 'turnover' # write the name of the construct X
defineY = 'job performance' # write the name of construct Y
report = 'full' # write 'short' or 'full' for the report format
run_query(purpose, defineX, defineY, report)

IV.measure,x.measure,y.measure,r3.Curators,r3.SampleSize,r3.SampleCountries,r3.SampleIndustries,r3.Reference,r3.PublicationTitle,r3.Theory,r3.Direction,r3.Moderator
mwork,turnover intentions,incidence of ocbs,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",60,"China, United States",Drug Retailing and Telecommunication,MECP16JAP,Journal of Applied Psychology,Expectancy theory,Unclear,N/A
organizational embeddedness,turnover intentions,incidence of ocbs,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",60,"China, United States",Drug Retailing and Telecommunication,MECP16JAP,Journal of Applied Psychology,Expectancy theory,Unclear,N/A
mwork,turnover intentions,incidence of ocbs,"Victor Chen, Meng Zhong, Wendy Long, Rosalyn Sandoval",60,"China, United States",Drug Retailing and Telecommunication,MECP16JAP,Journal of Applied Psychology,Expectancy theory,Unclear,N/A
